# VOD 영화 시청기록 데이터 - 마트 데이터 전처리

<br><hr>

### 00. 기본 설정

In [1]:
import pandas as pd

# 경고 메시지 출력 X
import warnings
warnings.filterwarnings("ignore")

<br><hr>

### 01. 데이터 불러오기

In [2]:
views = pd.read_csv('../data/movies_시청기록병합_asset정리안됨.csv', encoding='utf8')
mart = pd.read_csv('../data/영화_전처리.csv', encoding='utf8')
print(views.shape)  # 1188만 7938개
print(mart.shape)   # 5만 3882개

(11887938, 10)
(53874, 19)


In [5]:
# 컬럼 확인
print(views.columns)
print(mart.columns)

Index(['sha2_hash', 'asset', 'asset_nm', 'CT_CL', 'genre_of_ct_cl', 'use_tms',
       'disp_rtm', 'strt_dt', 'category', 'userId'],
      dtype='object')
Index(['full_asset_id', 'release_year', 'genre', 'vote_count', 'vote_average',
       'runtime', 'popularity', 'poster_path', 'original_title',
       'original_language', 'backdrop_path', 'adult', 'release_date',
       'movie_id', 'orgnl_cntry', 'name', 'actr_disp', 'director', 'overview'],
      dtype='object')


<br><hr>

### 02. 시청기록[asset] - VOD마트[full_asset_id] 비교

In [6]:
views_in_mart =  views[views['asset'].isin(mart['full_asset_id'])]
views_in_mart.shape  # 1188만 7938개 >> 1030만 6830개 (158만 1108개 삭제)

(10306830, 10)

*VOD mart와 시청기록의 asset id를 통일하기 위해, VOD 마트에서 같은 `name`-`movie_id`를 가진 영화들이 가진 `full_asset_id` 중 대푯값을 선정해 해당 영화의 asset_id를 통일시키고, 시청기록 데이터에서도 `full_asset_id` 리스트와 같은 값이 있는 영화들의 asset id도 그 영화의 대푯값으로 변경*

In [111]:
# 1. VODmart에서 대푯값 선정
# title과 overview를 기준으로 그룹화하여 첫 번째 full_asset_id를 대푯값으로 선택
representative_ids = (
    mart.groupby(["name", "movie_id"])["full_asset_id"]
    .first()  # 첫 번째 값을 대푯값으로 선정
    .reset_index()
    .rename(columns={"full_asset_id": "representative_asset_id"})
) 

In [112]:
# 2. VODmart에 대푯값 추가
mart = mart.merge(representative_ids, on=["name", "movie_id"], how="left")

In [113]:
# 3. 시청기록 데이터에서 대푯값으로 asset_id 변경
# full_asset_id와 대푯값 매핑 테이블 생성
id_mapping = mart.set_index("full_asset_id")["representative_asset_id"].to_dict()

KeyError: 'representative_asset_id'

In [53]:
# 시청기록의 asset 값을 대푯값으로 변경
views["asset"] = views["asset"].map(id_mapping)

In [54]:
mart.shape

(71039, 20)

#### *mart에서 같은 제목-대푯값 asset_id를 가진 데이터의 중복 제거*

In [63]:
vodmart_unique = mart.drop_duplicates(subset=["name", "movie_id", "representative_asset_id"])
vodmart_unique[vodmart_unique['name']=='올빼미']

,name,overview,orgnl_cntry,movie_id,release_date,adult,backdrop_path,original_language,original_title,poster_path,popularity,runtime,vote_average,vote_count,genre,release_year,full_asset_id,actr_disp,director,representative_asset_id
29694,올빼미,맹인이지만 뛰어난 침술 실력을 지닌 경수는 어의 이형익에게 그 재주를 인정받아 궁으...,대한민국,806128,2022-11-23,False,/qpMRsjLBI9tVL8T6kzgojbhfyyh.jpg,ko,올빼미,/i1pkFRi96SsIqat5vIoFHCwysv1.jpg,8.423,118.0,7.4,51.0,"스릴러, 미스터리",2022.0,cjc|M5165344LSVL72331901,"류준열,유해진,최무성,조성하,박명훈,김성철,안은진,조윤서",안태진,cjc|M5165344LSVL72331901


In [64]:
print(mart.shape)             # 7만 1039개
print(vodmart_unique.shape)   # 1만 4452개

(71039, 20)
(14452, 20)


#### *인덱스 재정렬 & 컬럼명 정리 & 사용하지 않을 컬럼 정리*

In [65]:
# 인덱스 초기화
vodmart_unique.reset_index(inplace=True, drop=True)
vodmart_unique.head(5)

,name,overview,orgnl_cntry,movie_id,release_date,adult,backdrop_path,original_language,original_title,poster_path,popularity,runtime,vote_average,vote_count,genre,release_year,full_asset_id,actr_disp,director,representative_asset_id
0,언니가 간다,"서른 살, 화려한 의상 디자이너를 꿈꿨지만 장 의상실에서 디자인 보다는 주로 잡일을...",대한민국,109507,2007-01-04,False,/unnhYUBP8DGw1nNdpiroc7OPiA0.jpg,ko,언니가 간다,/ajItSHWXsPdZCPFMBf413PZTDM0.jpg,3.043,112.0,7.0,9.0,"코미디, 로맨스",2007.0,cjc|M0003402LSVC35990401,"고소영,이범수,유건,조안,이중문,오미희,옥고운,오달수,김성연,윤종신,박선혜",김창래,cjc|M0003402LSVC35990401
1,삼거리 극장,할머니와 단 둘이 살던 소녀 소단. 활동 사진 보러 간다는 말만 남기고 사라져 버린...,대한민국,72994,2006-11-23,False,/dE2IDhFlzLi1S5GDuIRHunodl1K.jpg,ko,삼거리 극장,/jBpiv0ct1HnFXIFB49ZSqQ8vQp9.jpg,1.714,120.0,5.1,10.0,"공포, 코미디, 판타지",2006.0,cjc|M0007833LSVC36004501,"김꽃비,천호진,박준면,조희봉,황강록,김동기,백경인,윤오성,최우혁,이덕재,한애리,박영수",전계수,cjc|M0007833LSVC36004501
2,4.4.4.,도대체 누가? 유명한 배우이자 모델인 제니퍼 트리는 어느날 정체를 알 수 없는 누...,미국,14125,2007-03-01,False,/pmkccdphrm1tHIjBjGXipOtnfuE.jpg,en,Captivity,/zemQbUeZvRT22EmAJuLOCZhCDJQ.jpg,11.930,85.0,4.9,425.0,"범죄, 공포, 스릴러",2007.0,cjc|M0008465LSGM14346201,"엘리샤 커스버트,다니엘 길리스,마이클 하니,라즈 알론소,매기 데이몬,프루이트 테일러...",롤랑 조페,cjc|M0008465LSGM14346201
3,여고생시집가기,때는 서기 570년경. 평강공주의 열 여섯번째 생일을 축하하기 위해 궁전안에 모여 ...,대한민국,63243,2004-12-23,False,/iGtpImtRwGXRibSbUEilt25xSyT.jpg,ko,여고생 시집가기,/iYARJ0XADYiojW7TxNBDReoJ4Ue.jpg,3.461,103.0,4.3,6.0,"코미디, 로맨스",2004.0,cjc|M0017676LSGI69616001,"은지원,임은경,임성언,심이영,박노식,황재석,김희성,박정율,신효진,김경환,민성주,정다...",오덕환,cjc|M0017676LSGI69616001
4,B형 남자친구,"운명 같은 사랑을 믿는 여대생 하미는, 우연히 길거리에서 마주친 영빈에게 첫눈에 필...",대한민국,49206,2005-02-03,False,/4huJ8IwQdbtgJKIqkUVm5wDjdn8.jpg,ko,B형 남자친구,/mkuGXEcboQjKNWGbDvbIy5Dy3lf.jpg,3.475,96.0,6.3,10.0,코미디,2005.0,cjc|M0018751LSV795151801,"이동건,한지혜",최석원,cjc|M0018751LSV795151801


In [71]:
new = {
  "name" : "title", 
  "actr_disp" : "actors",
  "representative_asset_id":"asset_id"
}

# 컬럼명 변경 적용
df_renamed = vodmart_unique.rename(columns=new)

In [72]:
df_drop = df_renamed.drop("full_asset_id", axis=1)
df_drop.shape

(14452, 19)

In [73]:
df_drop.head(3)

,title,overview,orgnl_cntry,movie_id,release_date,adult,backdrop_path,original_language,original_title,poster_path,popularity,runtime,vote_average,vote_count,genre,release_year,actors,director,asset_id
0,언니가 간다,"서른 살, 화려한 의상 디자이너를 꿈꿨지만 장 의상실에서 디자인 보다는 주로 잡일을...",대한민국,109507,2007-01-04,False,/unnhYUBP8DGw1nNdpiroc7OPiA0.jpg,ko,언니가 간다,/ajItSHWXsPdZCPFMBf413PZTDM0.jpg,3.043,112.0,7.0,9.0,"코미디, 로맨스",2007.0,"고소영,이범수,유건,조안,이중문,오미희,옥고운,오달수,김성연,윤종신,박선혜",김창래,cjc|M0003402LSVC35990401
1,삼거리 극장,할머니와 단 둘이 살던 소녀 소단. 활동 사진 보러 간다는 말만 남기고 사라져 버린...,대한민국,72994,2006-11-23,False,/dE2IDhFlzLi1S5GDuIRHunodl1K.jpg,ko,삼거리 극장,/jBpiv0ct1HnFXIFB49ZSqQ8vQp9.jpg,1.714,120.0,5.1,10.0,"공포, 코미디, 판타지",2006.0,"김꽃비,천호진,박준면,조희봉,황강록,김동기,백경인,윤오성,최우혁,이덕재,한애리,박영수",전계수,cjc|M0007833LSVC36004501
2,4.4.4.,도대체 누가? 유명한 배우이자 모델인 제니퍼 트리는 어느날 정체를 알 수 없는 누...,미국,14125,2007-03-01,False,/pmkccdphrm1tHIjBjGXipOtnfuE.jpg,en,Captivity,/zemQbUeZvRT22EmAJuLOCZhCDJQ.jpg,11.930,85.0,4.9,425.0,"범죄, 공포, 스릴러",2007.0,"엘리샤 커스버트,다니엘 길리스,마이클 하니,라즈 알론소,매기 데이몬,프루이트 테일러...",롤랑 조페,cjc|M0008465LSGM14346201


In [74]:
df_drop.to_csv('../data/movies_mart_14452.csv', encoding='utf8', index=False)

#### *시청기록 데이터*

- 열 삭제 (sha2_hash, asset_nm, CT_CL, category)

In [119]:
# 열 삭제
views_drop = views_in_mart.drop(['sha2_hash', 'asset_nm', 'CT_CL', 'category'], axis=1)
print(views_drop.shape)
views_drop.head(1)

(10387903, 4)


,asset,use_tms,strt_dt,user_id
0,cjc|M5165334LSVL72332501,1301.0,2.023112e+13,user000001


In [121]:
new_col = {
  "asset" : "asset_id", 
  "strt_dt": "date"
}

# 컬럼명 변경 적용
views_rename = views_drop.rename(columns=new_col)
print(views_rename.shape)
views_rename.head(1)

(10387903, 4)


,asset_id,use_tms,date,user_id
0,cjc|M5165334LSVL72332501,1301.0,2.023112e+13,user000001


#### *마트에서 런타임 가져와서 시청기록에 추가*

In [129]:
vod_runtime = df_drop[["asset_id", "runtime"]]

In [131]:
print(df_drop.shape)
df_drop['asset_id'].value_counts()

(14452, 19)


asset_id
cjc|M5151815LSGL39797301    6
cjc|M4988458LSVI11194801    5
cjc|M4665590LFOA69771901    4
cjc|M4172280LSGJ60259301    3
cjc|M4634198LSG988453701    3
                           ..
cjc|M5009850LSGJ35525901    1
cjc|M5009854LSVL13077801    1
cjc|M5009857LSGI59900801    1
cjc|M5009859LSGL22957801    1
cjc|M4401423LSGK67245801    1
Name: count, Length: 14312, dtype: int64

In [132]:
df_drop[df_drop['asset_id']=='cjc|M5151815LSGL39797301']

,title,overview,orgnl_cntry,movie_id,release_date,adult,backdrop_path,original_language,original_title,poster_path,popularity,runtime,vote_average,vote_count,genre,release_year,actors,director,asset_id
13927,위험한 관계,작가인 안토니아와 맥스는 3살짜리 딸 줄리아를 둔 행복한 부부. 더이상 바랄게 없는...,"남아프리카공화국,영국",7303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,NaN,NaN,드라마,1991.0,"저넬 티,펠릭스 로코",호세 하비에르 레예스,cjc|M5151815LSGL39797301
13928,위험한 관계,전직 군인인 블레이크는 에버린의 부탁으로 그녀의 남편 (미술품 위조단 두목) 레이몬...,미국,37658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,NaN,NaN,드라마,1988.0,"저넬 티,펠릭스 로코",호세 하비에르 레예스,cjc|M5151815LSGL39797301
13929,위험한 관계,항상 아버지의 부정을 보며 자라 사랑이란 느낌을 불결하게 느끼는 한 남자. 그에게 ...,이탈리아,42953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,드라마,1974.0,"저넬 티,펠릭스 로코",호세 하비에르 레예스,cjc|M5151815LSGL39797301
13930,위험한 관계,돈과 섹스가 결부되었을 땐 어느 누구도 믿을 수 없다!이는 셜리번 부인이 젊고 매력...,"미국,캐나다",45261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,드라마,1991.0,"저넬 티,펠릭스 로코",호세 하비에르 레예스,cjc|M5151815LSGL39797301
13931,위험한 관계,"외곽 지역의 한 모텔에서 밀회를 즐기는 두 남녀, 여자는 과거 국선변호사로서 이름을...",미국,44874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,드라마,1984.0,"저넬 티,펠릭스 로코",호세 하비에르 레예스,cjc|M5151815LSGL39797301
13932,위험한 관계,혜숙은 친구들과 등산을 갔다가 약혼자인 영훈의 친구 순철에게 겁탈당한 후 그와 결혼...,대한민국,2220,NaN,NaN,http://file.koreafilm.or.kr/thm/01/copy/00/23/...,NaN,NaN,http://file.koreafilm.or.kr/thm/02/00/02/38/tn...,NaN,98.0,NaN,NaN,드라마,1970.0,"저넬 티,펠릭스 로코",호세 하비에르 레예스,cjc|M5151815LSGL39797301


In [130]:
views_w_runtime = pd.merge(views_rename, vod_runtime, on="asset_id", how="left")
views_w_runtime.shape

(10518803, 5)

In [104]:
views_w_runtime['runtime'] = views_w_runtime['runtime'] * 60
views_w_runtime.head(3)

,asset_id,use_tms,date,user_id,runtime
0,cjc|M5165334LSVL72332501,1301.0,2.023112e+13,user000001,5280.0
1,cjc|M4136216LFOL77322201,163.0,2.023112e+13,user000002,7260.0
2,cjc|M4550583LSG861820101,5801.0,2.023112e+13,user000003,6300.0


In [106]:
views_w_runtime.shape

(54019818, 5)

In [105]:
views_w_runtime.to_csv('시청기록_열삭제ver.csv', encoding='utf8', index=False)

KeyboardInterrupt: 